In [ ]:
import praw
import pandas as pd

client_id = 'CLIENT_ID'
client_secret = 'CLIENT_SECRET'
user_agent = 'scraper_demo_v1'

reddit = praw.Reddit(client_id=client_id,
                     client_secret=client_secret,
                     user_agent=user_agent)


In [ ]:
# DEPRESSION

In [ ]:
import pandas as pd
import re
from tqdm import tqdm

query = ("depresie OR anxietate OR panica OR plans OR suferinta OR singuratate OR dezamagire"
         " OR frica OR \"probleme in familie\" OR \"ganduri negre\" OR PTSD"
         "OR tristete OR sinucidere OR suicidal OR angoasa")

subreddits = [
    "Roumanie", "moldova", 
    'romania', 'CasualRO', "WomenRO", "Men_RO", 
    "bucuresti", "brasov", "cluj", "iasi"
]

emotion_keywords = [
    'depresie', 'anxietate', 'panica', 'plans', 'suferinta', 'singuratate', 'dezamagire',
    'frica', 'probleme', 'familie', 'ganduri', 'negre', 'trist', 'oboseala', 'insomnie',
    'gol', 'inutil', 'vinovatie', 'neputinta', 'lipsa', 'speranta', 'durere', 'PTSD',
    'sinucidere', 'suicidal', 'angoasa', 'tristete'
]

def contains_emotion_keyword(sentence):
    return any(keyword.lower() in sentence.lower() for keyword in emotion_keywords)

for subreddit in subreddits:
    search_results = reddit.subreddit(subreddit).search(query, limit=1500)

    posts = []
    seen_sentences = set()

    for post in tqdm(search_results):
        if len(post.selftext.split()) > 9:
            sentences = re.split(r'[.!?\n]+(?:\s|$)', post.selftext)
            for sentence in sentences:
                cleaned_sentence = sentence.strip()
                if len(cleaned_sentence.split()) >= 3:
                    if cleaned_sentence not in seen_sentences:
                        if contains_emotion_keyword(cleaned_sentence):
                            posts.append([cleaned_sentence, post.title, post.subreddit.display_name, 'post'])
                            seen_sentences.add(cleaned_sentence)

        post.comment_sort = 'top'    
        post.comments.replace_more(limit=0)
        top_comments = post.comments[:15]   

        for comment in top_comments:
            comment_body = comment.body

            if len(comment_body.split()) > 9:
                sentences = re.split(r'[.!?\n]+(?:\s|$)', comment_body)
                for sentence in sentences:
                    cleaned_sentence = sentence.strip()
                    if len(cleaned_sentence.split()) >= 3:
                        if cleaned_sentence not in seen_sentences:
                            if contains_emotion_keyword(cleaned_sentence):
                                posts.append([cleaned_sentence, post.title, post.subreddit.display_name, 'comment'])
                                seen_sentences.add(cleaned_sentence)

    df = pd.DataFrame(posts, columns=['sentence', 'post_title', 'subreddit', 'source'])
    df.index = range(1, len(df) + 1)
    df.to_csv(f'secondVersion\\output_sentences_top10_comments_{subreddit}.csv', index_label='Index')

    print(f"Am salvat {len(posts)} propoziții filtrate (postări + top 10 comentarii) din subredditul: {subreddit}")


In [ ]:
import pandas as pd
import glob
import os

directory = 'secondVersion'
pattern = os.path.join(directory, 'output_sentences_top10*.csv')

file_list = glob.glob(pattern)

df_list = [pd.read_csv(file) for file in file_list]
merged_df = pd.concat(df_list, ignore_index=True)

merged_df = merged_df.reset_index(drop=True)
merged_df.to_csv('merged_output_sentences_top10.csv', index_label='Index')

In [ ]:
import pandas as pd
import math

df = pd.read_csv('merged_output_sentences_top10.csv')

n = len(df)
split_size = math.ceil(n / 3)

for i in range(3):
    start = i * split_size
    end = min((i + 1) * split_size, n)
    sub_df = df.iloc[start:end].reset_index(drop=True)
    sub_df.to_csv(f'merged_output_sentences_top10_part{i+1}.csv', index_label='Index')

In [ ]:
# NON-DEPRESSION

In [ ]:
import pandas as pd
import re
from tqdm import tqdm

query = ("fericire OR bucurie OR optimism OR zambet OR implinire OR concediu OR calatorie "
         "OR mancare OR dragut OR relaxare OR succes OR realizare OR distractie OR veselie "
         "OR satisfactie OR multumire OR sanatate OR entuziasm OR hobby OR sport OR prietenie "
         "OR iubire OR apreciere OR cadou OR sarbatoare OR petrecere OR vacanta OR familie "
         "OR amuzant OR comedie OR surpriza OR jocuri OR pasiune OR weekend OR placere "
         "OR liniste OR echilibru OR energie OR frumos OR natura OR aventura OR plimbare")

subreddits = [
    "Roumanie", "moldova", 
    'romania', 'CasualRO', "WomenRO", "Men_RO", 
    "bucuresti", "brasov", "cluj", "iasi"
]

emotion_keywords = [
    'fericire', 'liniste', 'bucurie', 'optimism', 'viata', 'frumoasa', 'zambet',
    'simpla', 'buna', 'linistita', 'zambete', 'multumire', 'implinire', 'concediu',
    'calatorie', 'mancare', 'dragut', 'relaxare', 'succes', 'realizare',
    'distractie', 'veselie', 'satisfactie', 'sanatate', 'entuziasm' 'amuzant',
    'comedie', 'surpriza', 'jocuri', 'pasiune', 'weekend', 'placere', 'echilibru',
    'energie', 'frumos', 'natura', 'aventura', 'plimbare', 'prieteni', 'iubire',
    'apreciere', 'cadou', 'sarbatoare', 'petrecere', 'vacanta', 'familie', 'hobby', 'sport'
]

ban_words = [
    'depresie', 'anxietate', 'panica', 'plâns', 'suferință', 'singurătate', 'dezamăgire',
    'frică', 'moarte', 'suicid', 'sinucidere',
    'sex', 'drog', 'iarba', 'alcool'
]

def contains_emotion_keyword(sentence):
    return any(keyword.lower() in sentence.lower() for keyword in emotion_keywords)

def contains_ban_word(sentence):
    return any(ban_word.lower() in sentence.lower() for ban_word in ban_words)

for subreddit in subreddits:
    search_results = reddit.subreddit(subreddit).search(query, limit=1500)

    posts = []
    seen_sentences = set()

    for post in tqdm(search_results):
        if len(post.selftext.split()) > 9:
            sentences = re.split(r'[.!?\n]+(?:\s|$)', post.selftext)
            for sentence in sentences:
                cleaned_sentence = sentence.strip()
                if len(cleaned_sentence.split()) >= 3:
                    if cleaned_sentence not in seen_sentences:
                        if contains_emotion_keyword(cleaned_sentence) and not contains_ban_word(cleaned_sentence):
                            posts.append([cleaned_sentence, post.title, post.subreddit.display_name, 'post'])
                            seen_sentences.add(cleaned_sentence)

        post.comment_sort = 'top'
        post.comments.replace_more(limit=0)
        top_comments = post.comments[:15]

        for comment in top_comments:
            comment_body = comment.body
            if len(comment_body.split()) > 2:
                sentences = re.split(r'[.!?\n]+(?:\s|$)', comment_body)
                for sentence in sentences:
                    cleaned_sentence = sentence.strip()
                    if len(cleaned_sentence.split()) >= 3:
                        if cleaned_sentence not in seen_sentences:
                            if contains_emotion_keyword(cleaned_sentence) and not contains_ban_word(cleaned_sentence):
                                posts.append([cleaned_sentence, post.title, post.subreddit.display_name, 'comment'])
                                seen_sentences.add(cleaned_sentence)

    df = pd.DataFrame(posts, columns=['sentence', 'post_title', 'subreddit', 'source'])
    df.index = range(1, len(df) + 1)
    df.to_csv(f'secondVersionNonDepression\\output_sentences_top10_comments_{subreddit}.csv', index_label='Index')

    print(f"Am salvat {len(posts)} propoziții curate (pozitive) din subredditul: {subreddit}")


In [ ]:
import pandas as pd
import glob
import os

directory = 'secondVersionNonDepression'
pattern = os.path.join(directory, 'output_sentences_top10*.csv')

file_list = glob.glob(pattern)

df_list = [pd.read_csv(file) for file in file_list]
merged_df = pd.concat(df_list, ignore_index=True)

merged_df = merged_df.reset_index(drop=True)
merged_df.to_csv('merged_output_sentences_NON_DEPRESSION.csv', index_label='Index')

In [ ]:
import os

folder_path = 'reddit_depression'  
file_count = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
print(f"Number of files in '{folder_path}':", file_count)

In [ ]:
import os

folder_path = 'reddit_non_depression'  
file_count = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
print(f"Number of files in '{folder_path}':", file_count)